# Backpropagation

## We have to consider the following steps

## How to start now?

## Prepare your dataset

In [2]:
# Your code

## Building your neural network

In [1]:
# Your code

## Implement your loss function(s)

In [3]:
# Your code    

## Implement the training loop

In [4]:
# Your code

# Consider the following steps:
# 1) Loop through your training data
#   1. 1) Choose number of epochs (How often do you want to loop through your complete dataset?)
# 2) Forward the data through your network
# 3) Calculate the loss
# 4) Perform backpropagation with SGD and update the weights
#   4. 1) Choose a learning rate to update your weights
# Repeat 1, 2, 3, 4 until the training converges or maximum epochs are reached